# PyTorch - MNIST

## Resources
https://www.aiworkbox.com/lessons/load-mnist-dataset-from-pytorch-torchvision

## Code Reference
https://gist.github.com/kdubovikov/eb2a4c3ecadd5295f68c126542e59f0a

In [1]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [2]:
import pdb

In [3]:
import sys
print("Python Version:", sys.version)

Python Version: 3.6.5 |Anaconda, Inc.| (default, Apr 26 2018, 08:42:37) 
[GCC 4.2.1 Compatible Clang 4.0.1 (tags/RELEASE_401/final)]


In [4]:
print(torch.__version__)

0.4.1


In [5]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.autograd import Variable

In [6]:
import time

In [7]:
# download and transform train dataset
# train_check = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
#                                                    download=True,
#                                                         train=True))

# test_check = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
#                                                    download=True,
#                                                         train=False))
# print(len(train_check))
# print(len(test_check))

In [9]:
# Turn down for faster convergence
t0 = time.time()

# download and transform train dataset
train_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
                                                          download=True, 
                                                          train=True,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), 
                                           batch_size=10, 
                                           shuffle=True)

# download and transform test dataset
test_loader = torch.utils.data.DataLoader(datasets.MNIST('../mnist_data', 
                                                          download=True, 
                                                          train=False,
                                                          transform=transforms.Compose([
                                                              transforms.ToTensor(), # first, convert image to PyTorch tensor
                                                              transforms.Normalize((0.1307,), (0.3081,)) # normalize inputs
                                                          ])), 
                                           batch_size=10, 
                                           shuffle=True)

class CNNClassifier(nn.Module):
    """Custom module for a simple convnet classifier"""
    def __init__(self):
        super(CNNClassifier, self).__init__()
        self.conv1 = nn.Conv2d(1, 10, kernel_size=5)
        self.conv2 = nn.Conv2d(10, 20, kernel_size=5)
        self.dropout = nn.Dropout2d()
        self.fc1 = nn.Linear(320, 50)
        self.fc2 = nn.Linear(50, 10)
    
    def forward(self, x):
        # input is 28x28x1
        # conv1(kernel=5, filters=10) 28x28x10 -> 24x24x10
        # max_pool(kernel=2) 24x24x10 -> 12x12x10
        
        # Do not be afraid of F's - those are just functional wrappers for modules form nn package
        # Please, see for yourself - http://pytorch.org/docs/_modules/torch/nn/functional.html
        x = F.relu(F.max_pool2d(self.conv1(x), 2))
        
        # conv2(kernel=5, filters=20) 12x12x20 -> 8x8x20
        # max_pool(kernel=2) 8x8x20 -> 4x4x20
        x = F.relu(F.max_pool2d(self.dropout(self.conv2(x)), 2))
        
        # flatten 4x4x20 = 320
        x = x.view(-1, 320)
        
        # 320 -> 50
        x = F.relu(self.fc1(x))
        x = F.dropout(x, training=self.training)
        
        # 50 -> 10
        x = self.fc2(x)
        
        # transform to logits
        return F.log_softmax(x)

# create classifier and optimizer objects
clf = CNNClassifier()
opt = optim.SGD(clf.parameters(), lr=0.01, momentum=0.5)

loss_history = []
acc_history = []

def train(epoch):
    clf.train() # set model in training mode (need this because of dropout)
    
    # dataset API gives us pythonic batching 
    for batch_id, (data, label) in enumerate(train_loader):
        data = Variable(data)
        target = Variable(label)
        
        # forward pass, calculate loss and backprop!
        opt.zero_grad()
        preds = clf(data)
        loss = F.nll_loss(preds, target)
        loss.backward()
        loss_history.append(loss.data[0])
        opt.step()
        
        if batch_id % 1000 == 0:
            print(loss.data[0])

def test(epoch):
    clf.eval() # set model in inference mode (need this because of dropout)
    test_loss = 0
    correct = 0
    
    for data, target in test_loader:
        data = Variable(data, volatile=True) 
        target = Variable(target)
        
        output = clf(data)
        test_loss += F.nll_loss(output, target).data[0]
        pred = output.data.max(1)[1] # get the index of the max log-probability
        correct += pred.eq(target.data).cpu().sum()

    test_loss = test_loss
    test_loss /= len(test_loader) # loss function already averages over batch size
    accuracy = 100. * correct / len(test_loader.dataset)
    acc_history.append(accuracy)
    print("---------------------------------------")
    print("Epoch %d" % epoch)
    print('\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:.0f}%)\n'.format(
        test_loss, correct, len(test_loader.dataset),
        accuracy))

n_epochs = 5    
for epoch in range(0, n_epochs):
    #print("Epoch %d" % epoch)
    train(epoch)
    test(epoch)

run_time = time.time() - t0
print('Example run in : %.3f s' % run_time)
    
print("-----------------------")
print("the end")
print("-----------------------") 

/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:60: UserWarning: Implicit dimension choice for log_softmax has been deprecated. Change the call to include dim=X as an argument.
/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:82: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number
/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:86: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


tensor(2.3227)
tensor(0.2062)
tensor(0.6607)
tensor(0.4653)
tensor(1.0642)
tensor(0.6845)


/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:94: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/Users/reshamashaikh/anaconda3/lib/python3.6/site-packages/ipykernel_launcher.py:98: UserWarning: invalid index of a 0-dim tensor. This will be an error in PyTorch 0.5. Use tensor.item() to convert a 0-dim tensor to a Python number


---------------------------------------
Epoch 0

Test set: Average loss: 0.0750, Accuracy: 9765/10000 (97%)

tensor(0.0932)
tensor(0.1796)
tensor(0.0686)
tensor(0.4832)
tensor(0.0636)
tensor(0.2519)
---------------------------------------
Epoch 1

Test set: Average loss: 0.0620, Accuracy: 9812/10000 (98%)

tensor(0.0801)
tensor(0.3056)
tensor(0.3933)
tensor(0.9284)
tensor(0.1697)
tensor(0.0456)
---------------------------------------
Epoch 2

Test set: Average loss: 0.0558, Accuracy: 9827/10000 (98%)

tensor(0.0265)
tensor(0.5931)
tensor(0.2133)
tensor(0.5907)
tensor(0.2291)
tensor(0.0078)
---------------------------------------
Epoch 3

Test set: Average loss: 0.0545, Accuracy: 9815/10000 (98%)

tensor(0.0296)
tensor(0.0216)
tensor(0.0237)
tensor(0.2580)
tensor(0.0273)
tensor(0.5290)
---------------------------------------
Epoch 4

Test set: Average loss: 0.0460, Accuracy: 9860/10000 (98%)

Example run in             : 175.031 s
-------
the end
-------


https://am207.github.io/2018spring/wiki/ValidationSplits.html


In [ ]:
#train

In [ ]:
#mnist_trainset

In [ ]:
#!ls -a ./data

In [ ]:
#!ls -alt ./data/processed/